In [1]:
import pandas as pd
import json
import warnings
warnings.filterwarnings('ignore')
import re
import nltk
from nltk.corpus import stopwords
import string

from sklearn.externals import joblib
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
def cleanResume(resumeText):
    resumeText = re.sub('[%s]' % re.escape("""!"$%&'()*,-/:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText

In [4]:
def getData():

  resumeDataSet = pd.read_csv('/content/drive/My Drive/new_dataset_of_resume_skills1.csv')
  resumeDataSet['cleaned_resume_skills'] = ''
  resumeDataSet['cleaned_resume_skills'] = resumeDataSet.Resume.apply(lambda x: cleanResume(x.lower()))
  
  return resumeDataSet

In [5]:
# def encoding(resumeDataSet):
  
#   le = LabelEncoder()    
#   resumeDataSet['Category'] = le.fit_transform(resumeDataSet['Category'])
#   joblib.dump(le,'/content/drive/My Drive/labelEncoder.pkl')


In [6]:
# def vectorizing(requiredText):

#   word_vectorizer = TfidfVectorizer( sublinear_tf=True, stop_words='english')
#   word_vectorizer.fit(requiredText)
#   WordFeatures = word_vectorizer.transform(requiredText)

#   joblib.dump(word_vectorizer,'/content/drive/My Drive/wordVectorizer.pkl')
#   return WordFeatures


In [7]:
# def trainModel(X,Y):

#   x_train,x_test,y_train,y_test = train_test_split(X,Y,random_state=4,test_size=0.25)

#   model = KNeighborsClassifier(n_neighbors=6).fit(x_train, y_train)
#   prediction = model.predict(x_test)
#   print('Accuracy of KNN :- {:.2f}'.format(model.score(x_test, y_test)))
#   joblib.dump(model,'/content/drive/My Drive/knn.pkl')


In [8]:
def topKNeighbours(y, testData,k=3):

  model = joblib.load('/content/drive/My Drive/knn.pkl')
  le = joblib.load('/content/drive/My Drive/labelEncoder.pkl')

  top_k = set()     # Set of unique Neighbours
  i = 1             # Predicted value will always be added to the set

  predictedValue = model.predict(testData)
  invrValue = le.inverse_transform(predictedValue)[0]
  
  top_k.add(invrValue)
  dist, ind = model.kneighbors(testData)
  # print(dist[0])
  # print(ind[0])

  for i in range(k-1) :
      top_k.add(y[ind[0][i]])
  
  return top_k


In [10]:
if __name__ == "__main__":

  # Get data
  resumeDataSet = getData()  

  # Y == Categories without encoding
  y = resumeDataSet['Category']
  
  # # Cleaned Resume values
  # reqText = resumeDataSet['cleaned_resume_skills'].values
  
  # # Encoding the Categories
  # encoding(resumeDataSet)

  # # The encoded required category values
  # reqTarget = resumeDataSet['Category'].values

  # # For TF-IDF
  # wordFeatures = vectorizing(reqText)

  # # Training the model
  # trainModel(wordFeatures,reqTarget)

  testData = ['Angularjs, Python, JIRA, keras, Jenkins ']
  cleanTestData = [x.lower() for x in testData]
 
  # Transforming the test data to vector form
  wordVec = joblib.load('/content/drive/My Drive/wordVectorizer.pkl')
  wordFeatures_testData = wordVec.transform(cleanTestData)

  # Top K categories
  topK = topKNeighbours(y,wordFeatures_testData)
  print(topK)
  

{'Data Science', 'HR', 'DevOps Engineer'}
